Part One - Retreiving and Analysing mRNA and Protein Sequences of Human Cadherin-7 (25 marks)


- Task 1 - Search NCBI for the human Cadherin-7 gene entry, how many alternate transcripts does it have, what are their official accession ids, and how long are they in nucleotides (5 marks)

- Task 2 - Calculate the percentage composition of A, C, G, and T for each transcript and report the results in a table with the transcripts as rows and the percentages for each nucleotide as columns (5 marks)

- Task 3 - Translate the transcript sequences into protein sequences, report the length of the resulting proteins and state the most frequent amino acid in each one, if there are joint frequent ones name them all (5 marks)


- up to 6 marks for including one additional piece of analysis.

- up to 4 marks for exceptionally well organised and executed approach.


Possible extensions here might be to:

- look at other human Cadherin genes to see whether they also use alternate splicing to create different isoforms

- compare human Cadherin-7 to orthologs in other species

- write a short summary of what Cadherins do

- look at the structure of the Cadherin protein family

- comment on the difference in structure between the different Cadherin-7 isoforms

# Task 1

not gonna write code for this one I don't think!

Searching up "Homo Sapiens Cadherin-7" in Genes database, clicking through to "NCBI reference sequences" then tried the new "Transcript table" function which was super clear and helpful as it already included the length in nucleotides!

4 alternate transcripts (accession number : length (nt)):
- NM_004361.5 : 12136
- NM_033646.4 : 12126
- NM_001317214.3 : 3407
- NM_001362438.2 : 12938

In [27]:
# TASK 2


"""_Methodology_:
Using the accession number obtained in task 1, I fetch each entry matching the acession ID's fromt he nucleotide DB, then read then genbank format.
I then extract the sequence from this entry using location info provided by the gene feature (location for this is just the whole sequence).
I then count the numbers of A,C,G,T and divide each by the sequence length (also provided by the genbank record) to get a fraction that they make up.
I then print this out (along with ID and Sequence_length), to be recorded in a table in my report document.
"""

#%pip install biopython
from Bio import Entrez
Entrez.email = "s2055618@ed.ac.uk"
from Bio import SeqIO

accession_numbers = ["NM_004361.5", "NM_033646.4", "NM_001317214.3", "NM_001362438.2"]

for i in range(len(accession_numbers)):
    handle = Entrez.efetch(db="nucleotide", id=accession_numbers[i], rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    
    if record.features:
        for feature in record.features:
            # type=cds == coding sequence, however we want the whole thing! or do I? (so using the gene feature)
            if feature.type == "gene":
                current_sequence = feature.location.extract(record).seq
                print('ID:', accession_numbers[i])
                print(current_sequence)
                seq_len = len(current_sequence)
                a_count = current_sequence.count('A')
                c_count = current_sequence.count('C')
                g_count = current_sequence.count('G')
                t_count = current_sequence.count('T')
                # printing the sequence length is just for peace of mind :)
                print("nt sequence length:", seq_len)
                print("A's:", a_count, "C's:", c_count, "G's:", g_count, "T's:", t_count)
                a_pct = (a_count / seq_len) * 100
                c_pct = (c_count / seq_len) * 100
                g_pct = (g_count / seq_len) * 100
                t_pct = (t_count / seq_len) * 100
                print("%A =", a_pct, "| %C =", c_pct, "| %G =", g_pct, "| %T =", t_pct, "\n")
            

ID: NM_004361.5
AGTCTGCCCCGCGCGCGGAGCTGCGCGCACTGGGTCCCCAAGAGCCCGCGGGCGTCCGGCAGCCGAGCGCACGTTCTTTCGGATGCACACGCCCGGGTCCCTGGCGTCTGACGCCGTGGGGAGGGCAGCGAGGCCCCAGGTACTTACTACACCATTCTTTGGCGAAGGCTATTCAGCAGTGGTGACCTCTTCCAATCCAACACTCTACAGATTATTATCTCTGGACTCCCAGCTGACACCCTGCCGGAGGCAAGAGCTACTAAGCCAACTGGAACTGTGCCTTTTCTCTTGTCAAGGTTTTTTTCTTACACAGGAAAAAGAAAGAAAAAAAAAAGATGAAGTTGGGCAAAGTGGAGTTCTGCCATTTTCTGCAGCTAATAGCTCTTTTCCTGTGTTTTTCTGGGATGAGTCAAGCAGAACTCTCAAGGTCCAGATCAAAGCCCTATTTCCAATCAGGGAGGTCCCGGACCAAGCGCAGCTGGGTGTGGAATCAGTTCTTTGTGCTGGAGGAATACATGGGTTCAGACCCCCTCTATGTAGGAAAGCTTCACTCTGATGTTGATAAAGGAGATGGTTCCATCAAATACATCTTGTCAGGCGAAGGGGCAAGTTCCATTTTCATTATTGATGAGAACACTGGGGATATTCATGCCACCAAGAGACTGGATCGTGAGGAGCAGGCCTACTACACGCTCCGAGCTCAAGCGCTGGATAGGCTCACCAACAAACCCGTGGAGCCCGAGTCGGAGTTTGTCATCAAAATTCAGGATATCAACGACAATGAACCCAAATTTTTGGATGGCCCATACACGGCAGGAGTTCCCGAAATGTCTCCCGTGGGGACCTCAGTGGTACAAGTGACAGCGACGGATGCTGATGATCCTACATATGGCAACAGTGCCAGAGTGGTCTACAGTATTCTGCAAGGACAGCCGTACTTCTCAGTGGAGCCAAAGACAGGAGTCATCAAGACTGCCCTTCCAA

In [42]:
# Task 3

# method fairly straightforward, as it's very similar to the previous one except I translate the nt sequences into protein sequences!
# except i impl. a really awesome count_amino_acids function thats actually efficient bc im the GOAT (ok so its not that efficient with the sort there...)
# ran into trouble with getting lots of asterisks in my sequence, on further research it looks like if my protein contains these asterisks then the translation
# may not be accurate, so going forward I will try to get the FASTA format rather than genbank

accession_numbers = ["NM_004361.5", "NM_033646.4", "NM_001317214.3", "NM_001362438.2"]

# NOTE: GETTING A WARNING WHEN TRANSLATING THE FIRST TRANSCRIPTION AS IT IS NOT A  MULTIPLE OF THREE
# TODO: FIGURE OUT HOW TO ADD PADDING TO THE END?

# NOTE: THERE ARE ATERISKS APPEARING IN MY PROTEIN SEQUENCE
# TODO: FIGURE OUT WHAT THEY ARE AND IF I NEED TO GET RID OF THEM
# RESOLUTION: OK, THEY ARE THE RESULT OF A NOT VERY GOOD PROTEIN TRANSLATION, MEANS THE TRANSLATION MAY NOT BE ACCURATE,
# SO GOING FORWARD I WILL USE PROTEIN SEQUENCES FASTA FORMAT.

def count_amino_acids(protein_seq):
    aa_counts = {}
    for i in range(len(protein_seq)):
        curr_amino_acid = protein_seq[i]
        if curr_amino_acid in aa_counts:
            aa_counts[curr_amino_acid] += 1
        else:
            aa_counts.update({curr_amino_acid : 1})
    # return the amino acid counts dictionary sorted by value (note: the - (negative) sign before the value (item[1]) sorts in desc order)
    return {k: v for k, v in sorted(aa_counts.items(), key=lambda item: -item[1])}

for i in range(len(accession_numbers)):
    handle = Entrez.efetch(db="nucleotide", id=accession_numbers[i], rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    
    if record.features:
        for feature in record.features:
            # type=cds == coding sequence, however we want the whole thing! (so using the gene feature)
            if feature.type == "gene":
                current_sequence = feature.location.extract(record).seq
                print('ID:', accession_numbers[i])
                current_protein_sequence = current_sequence.translate()
                print(current_protein_sequence)
                print("protein sequence length:", len(current_protein_sequence))
                current_proteins_list = current_protein_sequence.split('*')
                print(current_proteins_list)
                for p in current_proteins_list:
                    p_length = len(p)
                    print("protein length =", p_length)
                    if p_length != 0:
                        aa_count = count_amino_acids(p)
                        print(aa_count)
                        print("top amino acid =", list(aa_count)[0], "\n")
                
                
                

ID: NM_004361.5
SLPRARSCAHWVPKSPRASGSRAHVLSDAHARVPGV*RRGEGSEAPGTYYTILWRRLFSSGDLFQSNTLQIIISGLPADTLPEARATKPTGTVPFLLSRFFSYTGKRKKKKR*SWAKWSSAIFCS**LFSCVFLG*VKQNSQGPDQSPISNQGGPGPSAAGCGISSLCWRNTWVQTPSM*ESFTLMLIKEMVPSNTSCQAKGQVPFSLLMRTLGIFMPPRDWIVRSRPTTRSELKRWIGSPTNPWSPSRSLSSKFRISTTMNPNFWMAHTRQEFPKCLPWGPQWYK*QRRMLMILHMATVPEWSTVFCKDSRTSQWSQRQESSRLPFQTWIERLKTSICLSFRQRIWLVKMEDCQELHQSL*P*LMSTIIHLAFLEGLINITSQSHYL*PQLWPELKLLMQILELMLKWSTRLWMVMVWAFLRFLLTKKPRKESLLYRRSWILKPKQVTRYG*KLQIKMPTLAF*AWVRSVTRQL*R*LWKM*MSPLCSLHPCTLWRCRKLPRLGISLAL*QLMTQILPIAL*GTQLTETQTWRDTSILMPTVGSSQLPSLWIERQMLFTISQSLQWRARIHLK*EEAMWPSLYLTSMITPLNLPWTMRPPSVKMPSRGRLSRKSVLWIKMSHPMDTSFTSA*QRMQQITTTFH*KITKTTQPQY*PGETASGDRNNQFTICQFSLWTVDLPHLAAPTPSPSACVTVMLTA*PRPAMQRPMSYLLASVQEP**PYSPVS*HYWC*SSLSSL*EDGKKSPLFLTKKETSEKIL*DTMTRAGERRTRKRLTWLH*ETSTSSETPRPGGM*LQKFNS*VDQLLKASQIMSSLGNLFGKD*KKPMLILVLLLMTPCRHMLLKEMAQLLNHSAL*IPSAQTLIRTMTT*VTGDLALNDSRTCMGLAKRVCTHSLGTLIRNVLKKK*QQKIK*NEIK**TTTYRKQELPLLETDGCKYFSIFNCLDFCLGEAYLH*TYLKDCTDHRL*AFEGFLIKINAQ

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\Bio\Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


ID: NM_033646.4
LGQSEEIERAKLSSKKTVSSD*FYDFAFMPQELN*SRLSRQAKVLTTPFFGEGYSAVVTSSNPTLYRLLSLDSQLTPCRRQELLSQLELCLFSCQGFFLTQEKERKKKDEVGQSGVLPFSAANSSFPVFFWDESSRTLKVQIKALFPIREVPDQAQLGVESVLCAGGIHGFRPPLCRKASL*C**RRWFHQIHLVRRRGKFHFHY**EHWGYSCHQETGS*GAGLLHAPSSSAG*AHQQTRGARVGVCHQNSGYQRQ*TQIFGWPIHGRSSRNVSRGDLSGTSDSDGC**SYIWQQCQSGLQYSARTAVLLSGAKDRSHQDCPSKHG*RG*RPVFACHSGKGYGWSKWRTVRNYISHCDPN*CQR*STSLSSKVLSI*RPRVITCSLSCGQN*SC*CRYWS*C*NGVQDCGW*WFGHF*DFC*QRNPGRNHYYTEGAGF*SQNKLHATDRSCK*RCRPSLSELGSVQ*HDNCEDNCGRCR*APCVLFTLVPYGGVGSYPGWEYHWHCSSS*PRFFQ*PCEVLN*QKHRLGEILQY*CQQWGHHNCQVFGSRDKCYSQYHSPCNGEPESISSRKRLCGHHYT*HQ**RP*ICHGL*DHRL*KCPAGAGYPENQCCG*R*AIQWTPVLLQLNNGCNK*PQLFIER*QRQHSLNTDQEKRLPETGTISLLSANFHCGQWISLT*QHQHPHHPRV*L*C*RRSPDLQCRGLCPTCWPQYRSPDSHTRLCLDIIGVDPPYRHYEKTEKRAPYF*RRKRHQRKYCEIR*RGRGRGGHGSV*HGCTEKPQRHPRHQDPEGCDSRNSIPESTSF*KHPR*CHL*GIYLGKIKRSRC*SWCSSL*LPADICF*RKWLSC*ITQLFRFHQLKL*SEL*LPK*LGTSL*TTRGHVWDWPREFVLIALEP*FEMY*RKSNSKK*NKMK*NNKPLHTENKNSPCWRQMVVNISPFLIV*ISALVRHIFIRLI*RTALTTDSEHLKVF**K*MLSGL*